In [1]:
import os
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

2025-05-05 01:03:40.892168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746407021.123528      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746407021.191078      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_and_split_image_data(
    data_dir: str,
    test_size: float = 0.1,
    val_size: float = 0.2,
    batch_size: int = 32,
    img_size: tuple = (120, 160),
    random_state: int = 42
):
    """
    Load images from class-named subfolders, exclude 'Sample...' folder, preprocess, and split.

    Returns tf.data.Dataset objects for image-only data.
    """

    # find class directories
    class_dirs = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))
                  and not d.startswith("Sample")]
    label_map = {name: idx for idx, name in enumerate(sorted(class_dirs))}

    paths, labels = [], []
    for cls in class_dirs:
        folder = os.path.join(data_dir, cls)
        imgs = glob.glob(os.path.join(folder, "*.jpg")) + glob.glob(os.path.join(folder, "*.png"))
        paths.extend(imgs)
        labels.extend([label_map[cls]] * len(imgs))
    paths = np.array(paths)
    labels = np.array(labels)

    X_temp, X_test, y_temp, y_test = train_test_split(
        paths, labels, test_size=test_size, stratify=labels, random_state=random_state
    )
    val_frac = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_frac, stratify=y_temp, random_state=random_state
    )

    def process_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

    def make_ds(paths, labels, shuffle=False):
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))
        ds = ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
        if shuffle:
            ds = ds.shuffle(buffer_size=len(paths), seed=random_state)
        return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    train_ds = make_ds(X_train, y_train, shuffle=True)
    val_ds   = make_ds(X_val,   y_val,   shuffle=False)
    test_ds  = make_ds(X_test,  y_test,  shuffle=False)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = load_and_split_image_data("../dataset/Thermal Camera Images")

2025-05-05 01:03:56.481912: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
def build_image_model(
    img_shape=(120,160,3),
    conv_filters=(32, 64 , 128),
    layer_dropout = 0.5,
    dense_units=64,
    output_units=4,
    lr=1e-4
):
    """
    Image-only CNN classifier.
    """
    inp = tf.keras.Input(shape=img_shape, name="image_input")
    x = tf.keras.layers.Conv2D(conv_filters[0], 3, activation="relu", padding="same", use_bias=False)(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(conv_filters[1], 3, activation="relu", padding="same",use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(conv_filters[2], 3, activation="relu", padding="same",use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(dense_units, activation="relu")(x)
    x = tf.keras.layers.Dropout(layer_dropout)(x)
    out = tf.keras.layers.Dense(output_units, activation="softmax", name="output")(x)

    model = tf.keras.Model(inp, out, name="image_only")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [5]:
model = build_image_model()
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_image_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[checkpoint]
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 965ms/step - accuracy: 0.5075 - loss: 1.1443
Epoch 1: val_accuracy improved from -inf to 0.25000, saving model to best_image_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.5081 - loss: 1.1432 - val_accuracy: 0.2500 - val_loss: 1.5486
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 965ms/step - accuracy: 0.6649 - loss: 0.8081
Epoch 2: val_accuracy did not improve from 0.25000
140/140 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.6650 - loss: 0.8078 - val_accuracy: 0.2500 - val_loss: 2.0582
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7187 - loss: 0.6860
Epoch 3: val_accuracy improved from 0.25000 to 0.39375, saving model to best_image_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.7187 - loss: 0.6860 - val_accuracy: 0.3938 - val_loss: 1.7384
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 976ms/step - accuracy: 0.7341 - loss: 0.6302
Epoch 4: val_accuracy improved from 0.39375 to 0.59688,

In [6]:
model = tf.keras.models.load_model('best_image_model.keras')
test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
print("test accuracy is ", test_accuracy)

test accuracy is  0.9437500238418579
